## NFL Combine Statistics Web Scraping

The purpose of this project is to provide functions and a notebook for scraping statistics for players participating in the NFL combine across a user-specified range of years. All data is scraped from pro-football-reference.com, using BeautifulSoup and Selenium.  

Statistics include combine metrics for players participating in the combine as well as their associated college football statistics. Combine metrics include height/weight, 40-yard dash, vertical jump, bench press reps, broad jump, 3-cone drill, 20-yard shuttle, and draft result. College football statistics include passing, rushing, receiving, and defensive statistics. As a demonstration, this script scrapes combine and college football statistics for all players participating in the NFL combine over the last 5 years (2019-2023).  

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

from nfl_combine_statistics_web_scraping import(
    get_soups,
    get_pfr_combines_stats,
    get_pfr_players_stats
)

pd.set_option('display.max_columns', None)

### Scraping 2019-2023 NFL Combine Metric Statistics

First, I will scrape the past five years (2019-2023) worth of NFL combine metric statistics. The resulting dataframe is indexed by combine year and uniquely-generated player id and has columns for each combine metric.

On pro-football-reference.com, one column in the combine metric statistics table, 'college', contains a hyperlink to a given player's college statistics. Rather than include the hyperlink text, the actual link has been extracted instead. 

Note: Most players only have valid values for a subset of combine metrics. All missing combine metric entries have been set to np.nan.

In [3]:
years = np.arange(2019, 2024)

urls = [f"https://aws.pro-football-reference.com/draft/{i}-combine.htm" for i in years]

soups = get_soups(urls)

combine_metrics_df = get_pfr_combines_stats(years, soups)

combine_metrics_df.head(3)

player pos        school_name   
combine_year player_id                                           
2019         0          Johnathan Abram   S  Mississippi State  \
             1               Paul Adams  OT           Missouri   
             2           Nasir Adderley   S           Delaware   

                                                                  college   
combine_year player_id                                                      
2019         0          https://www.sports-reference.com/cfb/players/j...  \
             1          https://www.sports-reference.com/cfb/players/p...   
             2                                                        NaN   

                       height  weight  forty_yd  vertical  bench_reps   
combine_year player_id                                                  
2019         0           5-11   205.0      4.45       NaN         NaN  \
             1            6-6   317.0      5.18      27.0        16.0   
             2            6-0   206.0       NaN       NaN         NaN   

                        broad_jump  cone  shuttle   
combine_year player_id                              
2019         0               116.0   NaN      NaN  \
             1               103.0  7.68     4.74   
             2                 NaN   NaN      NaN   

                                                           draft_info  
combine_year player_id                                                 
2019         0               Oakland Raiders / 1st / 27th pick / 2019  
             1                                                    NaN  
             2          Los Angeles Chargers / 2nd / 60th pick / 2019

In [4]:
combine_metrics_df.to_csv("datasets/Combine Metrics 2019-2023.csv")

### Scraping College Football Statistics from 2019-2023 NFL Combine Participants

For players participating in the last five drafts, I will use the links scraped from the combine metrics statistic tables (see above) to obtain their college football statistics. 

The resulting dataframe is a concatenation of passing, rushing/receiving, and defense college football statistics tables. The dataframe is indexed by player id (see above) and college football season year and has columns for each college football statistic (e.g., rush attempts, passing yards).

Note: Most players only have valid statistics for a subset of college football statistics. All missing college football statistic entries have been set to np.nan.

In [5]:
valid_link = (combine_metrics_df.loc[combine_metrics_df["college"].notna(), "college"])

links = valid_link.values
player_ids = valid_link.index.get_level_values(1)
 
soups = get_soups(links) 

college_stats_df = get_pfr_players_stats(player_ids, soups)

college_stats_df.head(3)

school_name conf_abbr class pos     g  tackles_solo   
player_id year_id                                                              
0         *2015              Georgia       SEC    FR   S   8.0          11.0  \
          *2017    Mississippi State       SEC    JR  DB  12.0          43.0   
          *2018    Mississippi State       SEC    SR   S  13.0          53.0   

                   tackles_assists  tackles_total  tackles_loss  sacks   
player_id year_id                                                        
0         *2015               14.0           25.0           1.5    0.0  \
          *2017               28.0           71.0           5.0    2.0   
          *2018               46.0           99.0           9.0    3.0   

                   def_int  def_int_yds  def_int_yds_per_int  def_int_td   
player_id year_id                                                          
0         *2015        0.0          0.0                  NaN         0.0  \
          *2017        0.0          0.0                  NaN         0.0   
          *2018        2.0          9.0                  4.5         0.0   

                   pass_defended  fumbles_rec  fumbles_rec_yds   
player_id year_id                                                
0         *2015              0.0          0.0              NaN  \
          *2017              5.0          0.0              NaN   
          *2018              5.0          1.0              NaN   

                   fumbles_rec_td  fumbles_forced  rush_att  rush_yds   
player_id year_id                                                       
0         *2015               NaN             0.0       NaN       NaN  \
          *2017               NaN             2.0       NaN       NaN   
          *2018               NaN             1.0       NaN       NaN   

                   rush_yds_per_att  rush_td  rec  rec_yds  rec_yds_per_rec   
player_id year_id                                                             
0         *2015                 NaN      NaN  NaN      NaN              NaN  \
          *2017                 NaN      NaN  NaN      NaN              NaN   
          *2018                 NaN      NaN  NaN      NaN              NaN   

                   rec_td  scrim_att  scrim_yds  scrim_yds_per_att  scrim_td   
player_id year_id                                                              
0         *2015       NaN        NaN        NaN                NaN       NaN  \
          *2017       NaN        NaN        NaN                NaN       NaN   
          *2018       NaN        NaN        NaN                NaN       NaN   

                   pass_cmp  pass_att  pass_cmp_pct  pass_yds   
player_id year_id                                               
0         *2015         NaN       NaN           NaN       NaN  \
          *2017         NaN       NaN           NaN       NaN   
          *2018         NaN       NaN           NaN       NaN   

                   pass_yds_per_att  adj_pass_yds_per_att  pass_td  pass_int   
player_id year_id                                                              
0         *2015                 NaN                   NaN      NaN       NaN  \
          *2017                 NaN                   NaN      NaN       NaN   
          *2018                 NaN                   NaN      NaN       NaN   

                   pass_rating  
player_id year_id               
0         *2015            NaN  
          *2017            NaN  
          *2018            NaN

In [6]:
college_stats_df.to_csv("datasets/College Football Statistics 2019-2023.csv")

Next, I will group by player name to (1) average each numeric statistic across all seasons played and (2) keep the numeric statistic in the most recent season played. I will also rename all columns in the pattern: "{statistic} _ {aggregation type}"

In [7]:
agg_college_stats_df = (college_stats_df
                        .select_dtypes("number")
                        .groupby("player_id")
                        .agg(["mean", lambda x: x.iloc[-1]])
                        .rename(columns={"mean": "avg", "<lambda_0>": "last_season"},
                                level=1))

agg_college_stats_df.columns = [stat+"_"+agg for stat, agg in agg_college_stats_df.columns]

agg_college_stats_df.head(3)

,g_avg,g_last_season,tackles_solo_avg,tackles_solo_last_season,tackles_assists_avg,tackles_assists_last_season,tackles_total_avg,tackles_total_last_season,tackles_loss_avg,tackles_loss_last_season,sacks_avg,sacks_last_season,def_int_avg,def_int_last_season,def_int_yds_avg,def_int_yds_last_season,def_int_yds_per_int_avg,def_int_yds_per_int_last_season,def_int_td_avg,def_int_td_last_season,pass_defended_avg,pass_defended_last_season,fumbles_rec_avg,fumbles_rec_last_season,fumbles_rec_yds_avg,fumbles_rec_yds_last_season,fumbles_rec_td_avg,fumbles_rec_td_last_season,fumbles_forced_avg,fumbles_forced_last_season,rush_att_avg,rush_att_last_season,rush_yds_avg,rush_yds_last_season,rush_yds_per_att_avg,rush_yds_per_att_last_season,rush_td_avg,rush_td_last_season,rec_avg,rec_last_season,rec_yds_avg,rec_yds_last_season,rec_yds_per_rec_avg,rec_yds_per_rec_last_season,rec_td_avg,rec_td_last_season,scrim_att_avg,scrim_att_last_season,scrim_yds_avg,scrim_yds_last_season,scrim_yds_per_att_avg,scrim_yds_per_att_last_season,scrim_td_avg,scrim_td_last_season,pass_cmp_avg,pass_cmp_last_season,pass_att_avg,pass_att_last_season,pass_cmp_pct_avg,pass_cmp_pct_last_season,pass_yds_avg,pass_yds_last_season,pass_yds_per_att_avg,pass_yds_per_att_last_season,adj_pass_yds_per_att_avg,adj_pass_yds_per_att_last_season,pass_td_avg,pass_td_last_season,pass_int_avg,pass_int_last_season,pass_rating_avg,pass_rating_last_season
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,11.00,13.0,35.666667,53.0,29.333333,46.0,65.00,99.0,5.166667,9.0,1.666667,3.0,0.666667,2.0,3.0,9.0,4.5,4.5,0.00,0.0,3.333333,5.0,0.333333,1.0,NaN,NaN,NaN,NaN,1.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.75,6.0,50.250000,24.0,48.500000,19.0,98.75,43.0,7.750000,3.0,1.875000,1.5,0.250000,0.0,7.5,0.0,30.0,NaN,0.25,0.0,1.500000,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,0.25,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.40,12.0,23.400000,39.0,28.600000,40.0,52.00,79.0,7.300000,14.5,2.100000,4.0,0.200000,0.0,0.0,0.0,0.0,NaN,0.00,0.0,1.000000,1.0,0.000000,0.0,NaN,NaN,NaN,NaN,1.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Merging 2019-2023 NFL Combine Metrics and College Football Statistics 

Finally, I will merge the combine statistics and the college statistics into one dataframe. 

In [8]:
integrated_df = combine_metrics_df.merge(agg_college_stats_df,
                                         how="left",
                                         left_on="player_id",
                                         right_on="player_id")

integrated_df.head(3)

,player,pos,school_name,college,height,weight,forty_yd,vertical,bench_reps,broad_jump,cone,shuttle,draft_info,g_avg,g_last_season,tackles_solo_avg,tackles_solo_last_season,tackles_assists_avg,tackles_assists_last_season,tackles_total_avg,tackles_total_last_season,tackles_loss_avg,tackles_loss_last_season,sacks_avg,sacks_last_season,def_int_avg,def_int_last_season,def_int_yds_avg,def_int_yds_last_season,def_int_yds_per_int_avg,def_int_yds_per_int_last_season,def_int_td_avg,def_int_td_last_season,pass_defended_avg,pass_defended_last_season,fumbles_rec_avg,fumbles_rec_last_season,fumbles_rec_yds_avg,fumbles_rec_yds_last_season,fumbles_rec_td_avg,fumbles_rec_td_last_season,fumbles_forced_avg,fumbles_forced_last_season,rush_att_avg,rush_att_last_season,rush_yds_avg,rush_yds_last_season,rush_yds_per_att_avg,rush_yds_per_att_last_season,rush_td_avg,rush_td_last_season,rec_avg,rec_last_season,rec_yds_avg,rec_yds_last_season,rec_yds_per_rec_avg,rec_yds_per_rec_last_season,rec_td_avg,rec_td_last_season,scrim_att_avg,scrim_att_last_season,scrim_yds_avg,scrim_yds_last_season,scrim_yds_per_att_avg,scrim_yds_per_att_last_season,scrim_td_avg,scrim_td_last_season,pass_cmp_avg,pass_cmp_last_season,pass_att_avg,pass_att_last_season,pass_cmp_pct_avg,pass_cmp_pct_last_season,pass_yds_avg,pass_yds_last_season,pass_yds_per_att_avg,pass_yds_per_att_last_season,adj_pass_yds_per_att_avg,adj_pass_yds_per_att_last_season,pass_td_avg,pass_td_last_season,pass_int_avg,pass_int_last_season,pass_rating_avg,pass_rating_last_season
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Johnathan Abram,S,Mississippi State,https://www.sports-reference.com/cfb/players/j...,5-11,205.0,4.45,NaN,NaN,116.0,NaN,NaN,Oakland Raiders / 1st / 27th pick / 2019,11.0,13.0,35.666667,53.0,29.333333,46.0,65.0,99.0,5.166667,9.0,1.666667,3.0,0.666667,2.0,3.0,9.0,4.5,4.5,0.0,0.0,3.333333,5.0,0.333333,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Paul Adams,OT,Missouri,https://www.sports-reference.com/cfb/players/p...,6-6,317.0,5.18,27.0,16.0,103.0,7.68,4.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nasir Adderley,S,Delaware,NaN,6-0,206.0,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles Chargers / 2nd / 60th pick / 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
integrated_df.to_csv("datasets/Combine Metrics and Aggregated College Football Statistics 2019-2023.csv")